In [1]:
from __future__ import absolute_import, division, print_function
# !pip install torch  matplotlib numpy torchvision
import matplotlib.pyplot as plt
import matplotlib.pylab as plb
import numpy as np
import time
import pandas as pd
# !pip install torch
import torch
from torch.utils import data

from torch.optim import lr_scheduler

from torchvision import datasets, models

from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch.optim as optim


from torch.autograd import Variable

import torch.nn as nn

from torch.utils.data import Dataset
from torch.utils.data import DataLoader
torch.set_printoptions(precision=4)
np.set_printoptions(precision=4)
import itertools
import gc
torch.autograd.set_detect_anomaly(True)


/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
def mem_clean():
    gc.collect()
    torch.cuda.empty_cache()
mem_clean()
def ch_gen(Size_area, Num_AP, Num_samples, PL_alpha=36., PL_const=0):
    ch_w_fading = []
    for i in range(Num_samples):
        loc = Size_area * (np.random.rand(Num_AP, 2) - 0.5)
        ch_w_temp_band = []
        dist_vec = loc.reshape(Num_AP, 1, 2) - loc
        dist_vec = np.linalg.norm(dist_vec, axis=2)
        dist_vec = np.maximum(dist_vec, 5)
        # print("start")
        # print(dist_vec)
        # print("end")
        pu_ch_gain_db = - PL_const - PL_alpha * np.log10(dist_vec)
        # print(pu_ch_gain_db)
        pu_ch_gain = 10 ** (pu_ch_gain_db / 10)
        # print(pu_ch_gain)
        final_ch = np.maximum(pu_ch_gain, np.exp(-60))

        ch_w_fading.append(np.transpose(final_ch))
    # print(ch_w_fading,"\n")
    return np.array(ch_w_fading)



def rand_alloc(num_sample, num_user, num_chan):
    mat_val = np.zeros((num_sample, num_user, num_chan))
    mv=mat_val
    for i in range(num_sample):
        tot_iter = num_chan ** num_user
        rand_sel = np.random.randint(tot_iter)
        for j in range(num_user):
            sel_ind = (rand_sel // num_chan ** j) % num_chan
            mat_val[i, j, sel_ind] = 1
            mv[i,j,sel_ind]=1
            if sel_ind > 0:
                mat_val[i,j,sel_ind-1]=0.005
            if sel_ind < num_chan-1:
                mat_val[i,j,sel_ind+1]=0.005
#     print(mat_val)
    return mat_val,mv


def opt_alloc(chan, num_user, num_chan):
    chan_ori = np.exp(chan * chan_std + chan_mean)
    chan_ori = np.triu(chan_ori, 1)
    tot_int_val = 10000
    tot_alloc = 0
    tot_iter = num_chan ** num_user
    for i in range(tot_iter):
        mat_val = np.zeros((num_user, num_chan))
        mv=mat_val
        for j in range(num_user):
            sel_ind = (i // num_chan ** j) % num_chan
            mat_val[j, sel_ind] = 1
            
        cur_int_val = np.sum(np.sum(np.matmul(chan_ori, mat_val) * mat_val, 1), 0)
        if cur_int_val < tot_int_val:
            tot_alloc = mat_val
            tot_int_val = cur_int_val


    return tot_alloc, tot_int_val


def heu_alloc(chan, num_user, num_chan):
    chan_ori = np.exp(chan * chan_std + chan_mean)
    chan_ori = np.triu(chan_ori, 1)
    chan_lower = 1000*np.tri(num_user, num_user)
    chan_temp  = chan_ori + chan_lower

    mat_val = np.zeros((num_user, num_chan))
    sel_val = 0
    chan_idx = 0
    round_count = True

    sel_idx_mat = np.zeros(num_user)

    ## Atleast two channels are assigned for each channel
    if num_user > num_chan * 2:
        for i in range(num_chan):
            result = np.where(chan_temp == np.amin(chan_temp))
            mat_val[result[0], i] = 1
            mat_val[result[1], i] = 1

            chan_temp[result[0], :] = 1000
            chan_temp[result[1], :] = 1000

            chan_temp[:, result[0]] = 1000
            chan_temp[:, result[1]] = 1000

            sel_idx_mat[result[0]] = 1
            sel_idx_mat[result[1]] = 1

        remain_usr = num_user - 2 * num_chan

        for i in range(remain_usr):
            cur_low_idx = 0
            cur_low_val = 100000
            idx_ch_temp = i % num_chan
            for j in range(num_user):
                mat_val_temp = mat_val.copy()
                if sel_idx_mat[j] == 0:
                    mat_val_temp[j, idx_ch_temp] = 1
                    temp_cur_val = np.sum(np.sum(np.matmul(chan_ori, mat_val_temp) * mat_val_temp, 1), 0)
                    if temp_cur_val < cur_low_val:
                        cur_low_idx = j
                        cur_low_val = temp_cur_val

            mat_val[cur_low_idx, idx_ch_temp] = 1
            sel_idx_mat[cur_low_idx] = 1

    else:
        sel_chan = 0
        remain_usr = num_user - 2 * sel_chan
        remain_ch = num_chan - sel_chan
        while remain_usr != remain_ch:
            result = np.where(chan_temp == np.amin(chan_temp))
            mat_val[result[0], sel_chan] = 1
            mat_val[result[1], sel_chan] = 1

            chan_temp[result[0], :] = 1000
            chan_temp[result[1], :] = 1000

            chan_temp[:, result[0]] = 1000
            chan_temp[:, result[1]] = 1000

            sel_idx_mat[result[0]] = 1
            sel_idx_mat[result[1]] = 1

            sel_chan += 1
            remain_usr = num_user - 2 * sel_chan
            remain_ch = num_chan - sel_chan

        for i in range(remain_usr):
            sel_flag = True
            for j in range(num_user):
                if (sel_idx_mat[j] == 0) & (sel_flag):
                    mat_val[j, sel_chan] = 1
                    sel_idx_mat[j] = 1
                    sel_flag = False
            sel_chan += 1
    mv = mat_val
    for i in range(mat_val.shape[0]):
        for j in range(mat_val.shape[1]):
            if mat_val[i][j]==1:
                if j > 0:
                    mat_val[i][j-1]=0.005
                if j < mat_val.shape[1]-1:
                    mat_val[i][j+1]=0.005
                
    tot_alloc = np.sum(np.sum(np.matmul(chan_ori, mat_val) * mv, 1), 0)
#     print(mat_val)
    return mat_val, tot_alloc

def cal_DL_ran(chan, mat_val, mv, chan_mean, chan_std):
    # Normalize channel gains
#     print(mat_val)
    chan_ori = np.exp(chan * chan_std + chan_mean)
    chan_triu = np.triu(chan_ori, 1)
    cur_int_val = np.zeros(chan.shape[0])
    int_alloc = np.matmul(chan_triu, mat_val) * mv
    int_alloc = np.sum(np.sum(int_alloc, axis=-1), axis=-1)
    cur_int_val += int_alloc    
    return cur_int_val

def cal_DL(chan, mat_val, chan_mean, chan_std):
    # Normalize channel gains
#     print(mat_val)
    _ , max_indices = torch.max(mat_val, dim=2, keepdim=True)
#         print(max_indices[0,0,0])
#         print(max_indices)
    out_RA_one_hot = torch.zeros_like(mat_val,device=device)
    out_RA_one_hot.scatter_(2, max_indices, 1)
    mv=out_RA_one_hot.clone()
    adjacent_update = torch.zeros_like(mat_val,device=device)

# Compute indices for updating adjacent elements
    update_indices_left = torch.clamp(max_indices - 1, min=0)
    update_indices_right = torch.clamp(max_indices + 1, max=mat_val.size(2) - 1)

    # Update adjacent elements using vectorized operations
    adjacent_update.scatter_(2, update_indices_left, 0.005)
    adjacent_update.scatter_(2, update_indices_right, 0.005)

    # Adjust updates for edge cases
    # If max_indices is at the beginning, set left update to 0
    adjacent_update[:, :, 0] *= (max_indices[:, :, 0] != 0).float()
    # If max_indices is at the end, set right update to 0
    adjacent_update[:, :, -1] *= (max_indices[:, :, 0] != (mat_val.size(2) - 1)).float()

    # Add the adjacent update to the one-hot encoded tensor
    out_RA_one_hot = out_RA_one_hot + adjacent_update
    out_RA_one_hot.requires_grad = True
    
    
    chan_ori = torch.exp(chan * chan_std + chan_mean)
    chan_triu = torch.triu(chan_ori, 1)
    cur_int_val = torch.zeros(chan.shape[0],device=device)
    int_alloc = torch.matmul(chan_triu, out_RA_one_hot) * mv
    int_alloc = torch.sum(torch.sum(int_alloc, axis=-1), axis=-1)
    cur_int_val += int_alloc
    return cur_int_val


# neural network definition
class Net(nn.Module):
    def __init__(self, num_user, num_chan, hidden_dim, num_l):
        super(Net, self).__init__()
        self.num_user = num_user
        self.num_chan = num_chan
        self.hidden_dim = hidden_dim
        self.num_l = num_l

        ## List of linear layers
        self.layers = nn.ModuleList()
        self.layers.append(nn.Linear(int((num_user) * (num_user-1) / 2), hidden_dim))
        for lll in range(num_l):
            self.layers.append(nn.Linear(hidden_dim, hidden_dim))

        ## List of dropout layers
        self.dropouts = nn.ModuleList()
        for lll in range(num_l):
            self.dropouts.append(nn.Dropout(0.1))

        ## List of Batch layers
        self.batches = nn.ModuleList()
        for lll in range(num_l):
            self.batches.append(nn.BatchNorm1d(hidden_dim))

        ## output for softmax and sigmoid
        self.out_RA = nn.Linear(hidden_dim, num_user * num_chan)

    def forward(self, x):
        z = torch.flatten(x, 1)  # flatten all dimensions except the batch dimension
        for (layer, dropout, batch_norm) in zip(self.layers, self.dropouts, self.batches):
              z = layer(z)
              z = batch_norm(z)
              z = F.relu(z)
              z = dropout(z)

        u = self.out_RA(z)
        u = u.view(-1, self.num_user, self.num_chan)
        out_RA = torch.softmax(u, dim=2)  # Use torch.softmax instead of F.softmax
        # print(out_RA_one_hot)
        return out_RA



def my_loss(mat_val, chan, chan_mean, chan_std):

    _ , max_indices = torch.max(mat_val, dim=2, keepdim=True)
#         print(max_indices[0,0,0])
#         print(max_indices)
    out_RA_one_hot = torch.zeros_like(mat_val,device=device)
    out_RA_one_hot.scatter_(2, max_indices, 1)
    mv=out_RA_one_hot.clone()
    adjacent_update = torch.zeros_like(mat_val,device=device)

# Compute indices for updating adjacent elements
    update_indices_left = torch.clamp(max_indices - 1, min=0)
    update_indices_right = torch.clamp(max_indices + 1, max=mat_val.size(2) - 1)

    # Update adjacent elements using vectorized operations
    adjacent_update.scatter_(2, update_indices_left, 0.005)
    adjacent_update.scatter_(2, update_indices_right, 0.005)

    # Adjust updates for edge cases
    # If max_indices is at the beginning, set left update to 0
    adjacent_update[:, :, 0] *= (max_indices[:, :, 0] != 0).float()
    # If max_indices is at the end, set right update to 0
    adjacent_update[:, :, -1] *= (max_indices[:, :, 0] != (mat_val.size(2) - 1)).float()

    # Add the adjacent update to the one-hot encoded tensor
    out_RA_one_hot = out_RA_one_hot + adjacent_update
    out_RA_one_hot.requires_grad = True

    chan_ori = torch.exp(chan*chan_std + chan_mean)
    chan_ori = torch.triu(chan_ori, diagonal=1)
    int_sum = torch.matmul(chan_ori, mat_val)*mat_val
    int_sum = torch.sum(torch.sum(int_sum, 2), 1)
    return torch.log(int_sum.mean())

# def my_loss(mat_val, chan, chan_mean, chan_std):
#     chan_ori = torch.exp(chan*chan_std + chan_mean)
#     chan_ori = torch.triu(chan_ori, diagonal=1)
#     int_sum = torch.matmul(chan_ori, mat_val)*mat_val
#     int_sum = torch.sum(torch.sum(int_sum, 2), 1)
#     return torch.log(int_sum.mean())




class ChanDataset(Dataset):
    def __init__(self, loc_val):
        super(ChanDataset, self).__init__()
        self.x_data = torch.Tensor(loc_val)
        self.x_data = torch.triu(self.x_data, diagonal=1)
        temp_val = []
        idx = torch.triu_indices(*self.x_data[0].shape, 1)
        for i in range(loc_val.shape[0]):
            temp_val.append(self.x_data[i, idx[0], idx[1]])
        self.y_data = torch.stack(temp_val, 0)

    def __len__(self):
        return len(self.x_data)

    def __getitem__(self, idx):
        x = torch.FloatTensor(self.x_data[idx])
        y = torch.FloatTensor(self.y_data[idx])
        return (x, y)





TOT_CCI_TOT = []

for jj in range(5):

    net_size=50
    num_user=8
    num_chan=5
#     net_size = 10*(jj+1)
#     print("net_size",net_size)
    num_user = 5 * (jj+1)
    print("num_user",num_user)
#     num_chan = 2+jj
#     print("num_chan",num_chan)
    tx_px = 10 * (3.0)


    tot_sample_tr = int(10 ** 4)
    tot_sample_te = int(10 ** 2)

    # loc_val_tr = ch_gen(net_size, num_user, tot_sample_tr, PL_alpha=38., PL_const=34.5)
    # loc_val_te = ch_gen(net_size, num_user, tot_sample_te, PL_alpha=38., PL_const=34.5)
    loc_val_tr = ch_gen(net_size, num_user, tot_sample_tr, PL_alpha=55., PL_const=98)
    loc_val_te = ch_gen(net_size, num_user, tot_sample_te, PL_alpha=55., PL_const=98)

    # print("loc_chan_tr",loc_val_tr)

    loc_val_tr_db = np.log(loc_val_tr)
    # print(loc_val_tr_db)
    # print("loc_chan_tr_db",loc_val_tr_db)
    # print(loc_val_tr_db)
    loc_val_te_db = np.log(loc_val_te)

    chan_mean = np.mean(loc_val_tr_db)
    chan_std = np.std(loc_val_tr_db)

    print(chan_mean)
    print(chan_std)
    loc_val_tr_norm = (loc_val_tr_db - chan_mean) / chan_std
    loc_val_te_norm = (loc_val_te_db - chan_mean) / chan_std

    tr_dataset = ChanDataset(loc_val_tr_norm)
    val_dataset = ChanDataset(loc_val_te_norm)

    train_dataloader = DataLoader(tr_dataset, batch_size=2048, shuffle=True)
    test_dataloader = DataLoader(val_dataset, batch_size=tot_sample_te, shuffle=False)

    device = 'cuda' if torch.cuda.is_available() else 'cpu'
#     device='cpu'
    net = Net(num_user, num_chan, 3000, 6).to(device)
    optimizer = torch.optim.Adam(net.parameters(), lr=0.1*1e-4)

    TOT_CCI_TEMP = []
    for i in range(300):
        
        net.train()
        cur_loss_tot = 0
        for index, (chan, chan_triu) in enumerate(train_dataloader):
            
            optimizer.zero_grad()
            chan_triu_gpu=chan_triu.to(device)
            chan_gpu=chan.to(device)
            
            mat_val=net(chan_triu_gpu)
#             start=time.time() 
            cur_loss = my_loss(mat_val, chan_gpu, chan_mean, chan_std)
#             print("time for 1 epochs = ", time.time()-start,"s")
            cur_loss_tot += cur_loss
            cur_loss.backward()
            optimizer.step()

            #if index % 100 == 0:
            #    print("index = ", i, "curloss", cur_loss_tot.detach() / (index + 1))
            #    print("")
        
        net.eval()
        if i%25 == 0:
            for index, (chan_val, chan_triu_val) in enumerate(test_dataloader):
                dl_sel = net(chan_triu_val.to(device))
                chan_np = chan_val.clone()
                
                dl_per = cal_DL(chan_np.to(device),dl_sel, chan_mean, chan_std)
                del dl_sel
                dl_per=dl_per.cpu().detach().numpy()
                print("DL per (during training) = ", 10*np.log10(np.mean(dl_per) / 2))

    ## Inference phase
    for index, (chan_val, chan_triu_val) in enumerate(test_dataloader):

        # opt_val_tot = 0
        # for jj in range(tot_sample_te):
        #     _, opt_val_temp = opt_alloc(chan_np[jj], num_user, num_chan)
        #     opt_val_tot += opt_val_temp
        # print("Opt per = ", 10*np.log10(opt_val_tot / 2 / tot_sample_te))
        # print("")
        # TOT_CCI_TEMP.append(10*np.log10(opt_val_tot / 2 / tot_sample_te))

        heu_val_tot = 0
        for jj in range(tot_sample_te):
            mat_val, heu_val_temp = heu_alloc(chan_np[jj], num_user, num_chan)
            # print(mat_val)
            heu_val_tot += heu_val_temp
        print("Heu per = ", 10*np.log10(heu_val_tot / 2 / tot_sample_te))
        # print("")
        TOT_CCI_TEMP.append(10*np.log10(heu_val_tot / 2 / tot_sample_te))


        rand_alloc_val,mv = rand_alloc(tot_sample_te, num_user, num_chan)
        # print(rand_alloc_val)
        rand_per = cal_DL_ran(chan_np, rand_alloc_val,mv, chan_mean, chan_std)
        print("Rand per = ", 10*np.log10(np.mean(rand_per) / 2))
        TOT_CCI_TEMP.append(10*np.log10(np.mean(rand_per) / 2))


        dl_sel = net(chan_triu_val.to(device))
        chan_np = chan_val.clone()
        # print(dl_sel.cpu().detach().numpy())
        dl_per = cal_DL(chan_np.to(device), dl_sel, chan_mean, chan_std)
        dl_per = dl_per.cpu().detach().numpy()
        print("DL per = ", 10 * np.log10(np.mean(dl_per) / 2))
        net.cpu()
        del net,mat_val,mv,chan_np
        mem_clean()
        TOT_CCI_TEMP.append(10 * np.log10(np.mean(dl_per) / 2))


        print("***" * 30)
        print("***" * 30)
        print("***" * 30)


    TOT_CCI_TOT.append(np.array(TOT_CCI_TEMP))


print("CCI")
print(np.array(TOT_CCI_TOT))



df=pd.DataFrame(TOT_CCI_TOT)
df.to_csv("user_data.csv")

num_user 25
-39.39352227186145
3.5095777971528506
DL per (during training) =  -134.29572381200924
DL per (during training) =  -136.03615481475765
DL per (during training) =  -137.24064686181958
DL per (during training) =  -137.7572457139872


In [ ]:
gc.collect()
torch.cuda.empty_cache()

In [ ]:
#dl_sel.cpu()
#del dl_sel
# net.cpu()
# del net

In [ ]:
def memory_stats():
    print(torch.cuda.memory_allocated()/1024**2)
    print(torch.cuda.memory_reserved()/1024**2)
memory_stats()